# Trabalho Final
Neste trabalho, vamos buscar o reconhecimento dos dígitos da Linguagem de Sinais. Para tal, vamos nos utilizar de Modelos conhecidos de Deep Learning e também nos aventurar na criação de próprios.

------

## Imports

In [1]:
import os
import numpy as np
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Dense, Flatten
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

import inf619utils

Using TensorFlow backend.


# Dataset
O dataset é composto por 10 classes (dígitos de 0 a 9) com aproximadamente 205 imagens por classe. 
O conjunto foi dividido em 60% para treinamento, 15% para validação e 20% para teste.
As imagens estão divididas em blocos por classe.

** IMPORTANTE NÃO ALTERAR O NOME/LOCAL DAS IMAGENS** 

In [2]:
datasetDir = "./Dataset"
datasetDirSplitted = "./DatasetSplitted"
nbClasses = 10
input_shape = (224,224,3)
input_size = (224,224)

train_files = {}
val_files = {}
test_files = {}

train_files, val_files, test_files = inf619utils.splitData(datasetDir, nbClasses)

In [3]:
#plot the images from imgList
def plotImagesFromBatch(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])

## Definição do modelo 

In [4]:
prev_model = ResNet50(input_shape=input_shape, classes=nbClasses, include_top=False)

In [54]:
top_model = Sequential()
top_model.add(Flatten())

model = Model(inputs=prev_model.input, outputs=top_model(prev_model.output))

AttributeError: 'Model' object has no attribute 'add'

In [52]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

# carregamento dos dados de treino, validação e test

In [5]:
batch_size = 32
epochs = 30

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './DatasetSplitted/train',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

validation_generator = val_datagen.flow_from_directory(
        './DatasetSplitted/val',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 1242 images belonging to 10 classes.
Found 309 images belonging to 10 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        datasetDirSplitted + '\\test',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 511 images belonging to 10 classes.


# Aplicação do Modelo no Conjunto de Testes

In [38]:
bottleneck_features_train = model.predict_generator(train_generator)

In [39]:
np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
np.save(open('bottleneck_features_train_y.npy', 'wb'), train_generator.classes)

In [40]:
bottleneck_features_val = model.predict_generator(validation_generator)

In [41]:
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_val)
np.save(open('bottleneck_features_validation_y.npy', 'wb'), validation_generator.classes)

In [42]:
bottleneck_features_test = model.predict_generator(test_generator)

In [43]:
np.save(open('bottleneck_features_test.npy', 'wb'), bottleneck_features_test)
np.save(open('bottleneck_features_test_y.npy', 'wb'), test_generator.classes)

In [44]:
del bottleneck_features_train, bottleneck_features_val, bottleneck_features_test

# Carregar variaves da ResNet

In [45]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
train_labels = np.load(open('bottleneck_features_train_y.npy','rb'))

X_train = np.reshape(train_data, (1, train_data.shape[0], train_data.shape[2]))

In [46]:
val_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
val_labels = np.load(open('bottleneck_features_validation_y.npy', 'rb'))

In [47]:
test_data = np.load(open('bottleneck_features_test.npy', 'rb'))
test_labels = np.load(open('bottleneck_features_test_y.npy', 'rb'))

In [63]:
X_train = np.reshape(train_data, (train_data.shape[0], train_data.shape[3]))
X_val = np.reshape(val_data, (val_data.shape[0], val_data.shape[3]))
X_test = np.reshape(test_data, (test_data.shape[0], test_data.shape[3]))

print(X_train.shape, X_val.shape, X_test.shape)

(1242, 2048) (309, 2048) (511, 2048)


# Treinar o modelo (Random Forest)

In [61]:
print(train_data.shape[0], train_data.shape[3])

1242 2048


# Treinar o modelo (SVM)

In [65]:
from sklearn import svm

clf = svm.SVC(kernel='rbf', decision_function_shape='ovo')
clf.fit(X_train, train_labels)

C:\Users\rafae\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [66]:
y_pred = clf.predict(X_val)

In [73]:
from sklearn.metrics import classification_report

report = classification_report(val_labels, y_pred)
print(report) 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        31
           3       0.00      0.00      0.00        31
           4       0.22      0.65      0.33        31
           5       0.47      0.71      0.56        31
           6       0.16      0.16      0.16        31
           7       0.00      0.00      0.00        31
           8       0.13      0.58      0.21        31
           9       0.00      0.00      0.00        30

   micro avg       0.21      0.21      0.21       309
   macro avg       0.10      0.21      0.13       309
weighted avg       0.10      0.21      0.13       309



# Gerar matrix de confusão

In [74]:
from sklearn.metrics import confusion_matrix

confusion_matrix(val_labels, y_pred)

array([[ 0,  0,  0,  0,  3,  0,  4,  0, 24,  0],
       [ 0,  0,  0,  0,  1,  0,  1,  0, 29,  0],
       [ 0,  0,  0,  0, 11,  1,  6,  0, 13,  0],
       [ 0,  0,  0,  0,  3,  9,  3,  0, 16,  0],
       [ 0,  0,  0,  0, 20,  5,  3,  0,  3,  0],
       [ 0,  0,  0,  0,  6, 22,  3,  0,  0,  0],
       [ 0,  0,  0,  0, 19,  1,  5,  0,  6,  0],
       [ 0,  0,  0,  0, 13,  0,  2,  0, 16,  0],
       [ 0,  0,  0,  0,  8,  2,  3,  0, 18,  0],
       [ 0,  0,  0,  0,  7,  7,  1,  0, 15,  0]], dtype=int64)